In [13]:
ls = sc.textFile('data/coupon150720.csv')

In [14]:
ls.take(5)

[u'79062005698500,1,MAA,AUH,9W,9W,56.79,USD,1,H,H,0526,150904,OK,IAF0',
 u'79062005698500,2,AUH,CDG,9W,9W,84.34,USD,1,H,H,6120,150905,OK,IAF0',
 u'79062005924069,1,CJB,MAA,9W,9W,60.0,USD,1,H,H,2768,150721,OK,IAA0',
 u'79065668570385,1,DEL,DXB,9W,9W,160.63,USD,2,S,S,0546,150804,OK,INA0',
 u'79065668737021,1,AUH,IXE,9W,9W,152.46,USD,1,V,V,0501,150803,OK,INA0']

In [15]:
ls.count()

1232662

In [16]:
ls.count()

1232662

In [17]:
ls.cache()

MapPartitionsRDD[12] at textFile at NativeMethodAccessorImpl.java:-2

In [18]:
ls.count()

1232662

In [19]:
ls.count()

1232662

In [20]:
ls.getNumPartitions()

3

In [21]:
ls.take(5)

[u'79062005698500,1,MAA,AUH,9W,9W,56.79,USD,1,H,H,0526,150904,OK,IAF0',
 u'79062005698500,2,AUH,CDG,9W,9W,84.34,USD,1,H,H,6120,150905,OK,IAF0',
 u'79062005924069,1,CJB,MAA,9W,9W,60.0,USD,1,H,H,2768,150721,OK,IAA0',
 u'79065668570385,1,DEL,DXB,9W,9W,160.63,USD,2,S,S,0546,150804,OK,INA0',
 u'79065668737021,1,AUH,IXE,9W,9W,152.46,USD,1,V,V,0501,150803,OK,INA0']

In [26]:
aerolineas = ls.map(lambda x: x.split(",")[5])

In [27]:
aerolineas.take(5)

[u'9W', u'9W', u'9W', u'9W', u'9W']

In [29]:
aerolineas.filter(lambda x: x == "IB").count()

26158

In [30]:
ls.map(lambda x: x.split(",")[5]).filter(lambda x: x == "IB").count()

26158

In [31]:
def getAerolineaDinero(l):
    elems = l.split(",")
    aerolinea = elems[5]
    dinero = float(elems[6])
    return (aerolinea, dinero)

In [37]:
getAerolineaDinero(ls.first())

(u'9W', 56.79)

In [35]:
dineros = ls.map(getAerolineaDinero)

In [39]:
dinerosIB = dineros.filter(lambda x: x[0] == "IB")

In [44]:
cantidades = dinerosIB.map(lambda x: x[1])

In [48]:
cantidades.sum()

2828044.4199999194

In [46]:
dineros.filter(lambda x: x[0] == "IB").map(lambda x: x[1]).stats()

(count: 26158, mean: 108.113939139, stdev: 168.011485542, max: 5239.93, min: 0.0)

In [49]:
ls.first()

u'79062005698500,1,MAA,AUH,9W,9W,56.79,USD,1,H,H,0526,150904,OK,IAF0'

In [52]:
def getRutaDinero(l):
    elems = l.split(",")
    origen = elems[2]
    destino = elems[3]
    dinero = float(elems[6])
    ruta = origen + "-" + destino
    return (ruta, dinero)

In [53]:
getRutaDinero(ls.first())

(u'MAA-AUH', 56.79)

In [54]:
rutas = ls.map(getRutaDinero)

In [55]:
rutas.take(3)

[(u'MAA-AUH', 56.79), (u'AUH-CDG', 84.34), (u'CJB-MAA', 60.0)]

In [58]:
rutas.filter(lambda x: x[1] < 7500).sortBy(lambda x: x[1],False).take(5)

[(u'BLA-MIA', 7490.33),
 (u'MIA-CCS', 7415.0),
 (u'MIA-CCS', 7415.0),
 (u'CCS-MIA', 7415.0),
 (u'MIA-CCS', 7415.0)]

In [59]:
filtrado = rutas.filter(lambda x: x[1] < 7500)

In [61]:
filtrado.cache()

PythonRDD[67] at RDD at PythonRDD.scala:43

In [62]:
filtrado.groupByKey().take(3)

[(u'SZF-IST', <pyspark.resultiterable.ResultIterable at 0x7f5df100e8d0>),
 (u'DME-RHO', <pyspark.resultiterable.ResultIterable at 0x7f5df100e810>),
 (u'KRK-OSL', <pyspark.resultiterable.ResultIterable at 0x7f5df100e510>)]

In [63]:
filtrado.groupByKey().mapValues(lambda vs: max(vs)).take(3)

[(u'SZF-IST', 289.83), (u'DME-RHO', 168.95), (u'KRK-OSL', 98.0)]

In [64]:
maximos = filtrado.groupByKey().mapValues(lambda vs: max(vs))

In [65]:
maximos.sortBy(lambda x: x[1], False).take(5)

[(u'BLA-MIA', 7490.33),
 (u'MIA-CCS', 7415.0),
 (u'CCS-MIA', 7415.0),
 (u'RUH-LAX', 7401.94),
 (u'DXB-MAA', 7343.27)]

In [67]:
filtrado.groupByKey().mapValues(lambda vs: sum(vs)/len(vs)) \
.sortBy(lambda x: x[1], False) \
.take(5) \

[(u'BLA-MEC', 5164.355),
 (u'MEC-BLA', 5164.355),
 (u'CCS-MIA', 4350.516296296297),
 (u'MIA-CCS', 4186.95),
 (u'NRT-PPT', 3751.5499999999997)]

In [68]:
ls.first()

u'79062005698500,1,MAA,AUH,9W,9W,56.79,USD,1,H,H,0526,150904,OK,IAF0'

In [73]:
def getRutaALDineroClase(l):
    elems = l.split(",")
    ruta = elems[2] + "-" + elems[3]
    al = elems[4]
    dinero = float(elems[6])
    clase = elems[9]
    return ((ruta, al, clase), dinero)

In [74]:
getRutaALDineroClase(ls.first())

((u'MAA-AUH', u'9W', u'H'), 56.79)

In [75]:
import math

In [96]:
def valorLimite(vs):
    logs = [math.log(v+1) for v in vs]
    media = sum(logs)/len(logs)
    desv = [(x-media)**2 for x in logs]
    var = sum(desv)/len(desv)
    desvStd = math.sqrt(var)
    return math.exp(media + 2*desvStd) - 1

In [98]:
valorLimite([2,2,2,2,2,2])

2.0000000000000004

In [99]:
rutas = ls.map(getRutaALDineroClase)

In [100]:
rutas.take(5)

[((u'MAA-AUH', u'9W', u'H'), 56.79),
 ((u'AUH-CDG', u'9W', u'H'), 84.34),
 ((u'CJB-MAA', u'9W', u'H'), 60.0),
 ((u'DEL-DXB', u'9W', u'S'), 160.63),
 ((u'AUH-IXE', u'9W', u'V'), 152.46)]

In [101]:
rutas.groupByKey().take(2)

[((u'TRD-BOO', u'SK', u'V'),
  <pyspark.resultiterable.ResultIterable at 0x7f5df0f75e50>),
 ((u'DPS-TMC', u'GA', u'N'),
  <pyspark.resultiterable.ResultIterable at 0x7f5df0f75f50>)]

In [103]:
def filtraCuponesExtremos(vs):
    vl = valorLimite(vs)
    f = [v for v in vs if v < vl]
    return f

In [104]:
grupos = rutas.groupByKey().mapValues(filtraCuponesExtremos)

In [117]:
def mediaGrupo(vs):
    if(len(vs) == 0):
        return 0.0
    else:
        return sum(vs)/len(vs)

In [118]:
grupos.mapValues(mediaGrupo).take(5)

[((u'TRD-BOO', u'SK', u'V'), 42.30263157894736),
 ((u'DPS-TMC', u'GA', u'N'), 60.7925),
 ((u'BEG-CDG', u'JU', u'K'), 0.0),
 ((u'SAV-CLT', u'US', u'N'), 0.0),
 ((u'DUS-AMS', u'KL', u'C'), 0.0)]

In [119]:
grupos.mapValues(mediaGrupo).sortBy(lambda x: x[1], False).take(5)

[((u'SYD-CGK', u'GA', u'G'), 6355194.0),
 ((u'COO-CDG', u'AF', u'J'), 538599.2),
 ((u'CDG-COO', u'AF', u'J'), 179628.3533333333),
 ((u'DKR-NBO', u'KQ', u'T'), 58253.593333333345),
 ((u'TSN-ICN', u'KE', u'Z'), 38655.45)]

In [120]:
grupos.take(2)

[((u'TRD-BOO', u'SK', u'V'),
  [42.76,
   48.67,
   42.76,
   42.76,
   42.76,
   48.67,
   42.76,
   48.67,
   42.76,
   42.76,
   42.76,
   42.76,
   42.76,
   29.55,
   42.76,
   42.76,
   29.55,
   42.76,
   42.76]),
 ((u'DPS-TMC', u'GA', u'N'), [59.15, 59.26, 62.38, 62.38])]

In [121]:
grupos = rutas.groupByKey()

In [122]:
grupos.take(2)

[((u'TRD-BOO', u'SK', u'V'),
  <pyspark.resultiterable.ResultIterable at 0x7f5df0f80810>),
 ((u'DPS-TMC', u'GA', u'N'),
  <pyspark.resultiterable.ResultIterable at 0x7f5df10723d0>)]

In [123]:
gruposFiltrados = grupos.filter(lambda x: len(x[1]) > 10)

In [125]:
r = gruposFiltrados.mapValues(filtraCuponesExtremos).mapValues(mediaGrupo)

In [127]:
r.sortBy(lambda x: x[1], False).take(5)

[((u'MIA-CCS', u'S3', u'S'), 7826.25),
 ((u'CCS-MIA', u'S3', u'B'), 5550.0),
 ((u'CCS-MAD', u'V0', u'Y'), 5418.098666666667),
 ((u'RUH-FRA', u'LH', u'F'), 4792.983),
 ((u'LHR-DOH', u'QR', u'F'), 4275.98923076923)]

In [128]:
ls.first()

u'79062005698500,1,MAA,AUH,9W,9W,56.79,USD,1,H,H,0526,150904,OK,IAF0'

In [132]:
ls.map(lambda x: x.split(",")[4]).distinct().count()

359

In [134]:
ls.map(lambda x: x.split(",")[7]).distinct().collect()

[u'USD']

In [140]:
ls.map(lambda x: x.split(",")[4]).distinct().saveAsTextFile("results.csv")